In [22]:
#collapse
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
#collapse
#export
from exp.nb_07 import *

## Fastai Layerwise Sequential Unit Variance (LSUV)
- This Post is based on the Notebok by the [Fastai Course Part2](https://course.fast.ai/)  
=> With LSUV the computer will figure out how to init our net proberly, and will initalize our network with unit Variance 

Getting the MNIST data and a CNN

[Jump_to lesson 11 video](https://course.fast.ai/videos/?lesson=11&t=235)

In [3]:
#collapse
x_train,y_train,x_valid,y_valid = get_data()

x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [4]:
#collapse_show
mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

In [5]:
#collapse
nfs = [8,16,32,64,64]

In [6]:
#collapse_show
class ConvLayer(nn.Module):
    def __init__(self, ni, nf, ks=3, stride=2, sub=0., **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=True)
        self.relu = GeneralRelu(sub=sub, **kwargs)
    
    def forward(self, x): return self.relu(self.conv(x))
    
    @property
    def bias(self): return -self.relu.sub
    @bias.setter
    def bias(self,v): self.relu.sub = -v
    @property
    def weight(self): return self.conv.weight

In [7]:
learn,run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

Now we're going to look at the paper [All You Need is a Good Init](https://arxiv.org/pdf/1511.06422.pdf), which introduces *Layer-wise Sequential Unit-Variance* (*LSUV*). We initialize our neural net with the usual technique, then we pass a batch through the model and check the outputs of the linear and convolutional layers. We can then rescale the weights according to the actual variance we observe on the activations, and subtract the mean we observe from the initial bias. That way we will have activations that stay normalized.

We repeat this process until we are satisfied with the mean/variance we observe.

Let's start by looking at a baseline:

In [8]:
#collapse_show
run.fit(2, learn)

train: [2.2064690625, tensor(0.2252, device='cuda:0')]
valid: [1.61256552734375, tensor(0.4537, device='cuda:0')]
train: [0.736686484375, tensor(0.7566, device='cuda:0')]
valid: [0.2495784423828125, tensor(0.9188, device='cuda:0')]


Now we recreate our model and we'll try again with LSUV. Hopefully, we'll get better results!

In [9]:
#collapse_show
learn,run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

Helper function to get one batch of a given dataloader, with the callbacks called to preprocess it.

In [10]:
#collapse_show
def get_batch(dl, run):
    run.xb,run.yb = next(iter(dl))
    for cb in run.cbs: cb.set_runner(run)
    run('begin_batch')
    return run.xb,run.yb

In [11]:
xb,yb = get_batch(data.train_dl, run)

We only want the outputs of convolutional or linear layers. To find them, we need a recursive function. We can use `sum(list, [])` to concatenate the lists the function finds (`sum` applies the + operate between the elements of the list you pass it, beginning with the initial state in the second argument).

In [12]:
#collapse_show
def find_modules(m, cond):
    if cond(m): return [m]
    return sum([find_modules(o,cond) for o in m.children()], [])

def is_lin_layer(l):
    lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)
    return isinstance(l, lin_layers)

In [13]:
#collapse_show
mods = find_modules(learn.model, lambda o: isinstance(o,ConvLayer))

In [14]:
#collapse_show
mods

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 )]

This is a helper function to grab the mean and std of the output of a hooked layer.

In [15]:
#collapse_show
def append_stat(hook, mod, inp, outp):
    d = outp.data
    hook.mean,hook.std = d.mean().item(),d.std().item()

In [16]:
#collapse_show
mdl = learn.model.cuda()

So now we can look at the mean and std of the conv layers of our model.

In [17]:
#collapse_show
with Hooks(mods, append_stat) as hooks:
    mdl(xb)
    for hook in hooks: print(hook.mean,hook.std)

0.44657835364341736 0.9164510369300842
0.6921732425689697 1.1961840391159058
0.5367525815963745 1.0832475423812866
0.6328485012054443 0.9635109901428223
0.43013903498649597 0.6254509091377258


We first adjust the bias terms to make the means 0, then we adjust the standard deviations to make the stds 1 (with a threshold of 1e-3). The `mdl(xb) is not None` clause is just there to pass `xb` through `mdl` and compute all the activations so that the hooks get updated. 

In [18]:
#collapse_show
def lsuv_module(m, xb):
    h = Hook(m, append_stat)

    while mdl(xb) is not None and abs(h.mean)  > 1e-3: m.bias -= h.mean
    while mdl(xb) is not None and abs(h.std-1) > 1e-3: m.weight.data /= h.std

    h.remove()
    return h.mean,h.std

We execute that initialization on all the conv layers in order:

In [19]:
#collapse_show
for m in mods: print(lsuv_module(m, xb))

(0.04071265459060669, 1.0)
(-0.019546931609511375, 1.0000001192092896)
(0.15097013115882874, 1.0)
(0.14636048674583435, 0.9999998211860657)
(0.3110796809196472, 0.9999998807907104)


Note that the mean doesn't exactly stay at 0. since we change the standard deviation after by scaling the weight.

Then training is beginning on better grounds.

In [20]:
#collapse_show
%time run.fit(2, learn)

train: [0.6585459375, tensor(0.7840, device='cuda:0')]
valid: [0.20176336669921874, tensor(0.9391, device='cuda:0')]
train: [nan, tensor(0.6914, device='cuda:0')]
valid: [nan, tensor(0.0991, device='cuda:0')]
CPU times: user 8.48 s, sys: 40.5 ms, total: 8.52 s
Wall time: 8.4 s


LSUV is particularly useful for more complex and deeper architectures that are hard to initialize to get unit variance at the last layer.

## Export

In [21]:
!python notebook2script.py 07a_lsuv.ipynb

Traceback (most recent call last):
  File "notebook2script.py", line 3, in <module>
    import json,fire,re
ModuleNotFoundError: No module named 'fire'
